In [ ]:
// Charger DEM
var dem= ee.Image('USGS/SRTMGL1_003').clip(CSP)

// Calcul de la pente et de l'aspect
var terrain = ee.Terrain.products(dem);
var slope = terrain.select('slope');
var aspect = terrain.select('aspect');

// 1-Choix de la fenetre temporelle

var start= '2019-05-01';
var end = '2019-07-31';

// 2-Charger la collection et filtre
var s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
   .filterBounds(CSP)
   .filterDate(start, end)
   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
   .median()
   .clip(CSP);
Map.centerObject(CSP)

// 3-Calcul des indices de vegetation

// NDVI
var ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI');

// EVI

var evi = s2.expression(
  '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
  {
    'NIR': s2.select('B8').multiply(0.0001),
    'RED': s2.select('B4').multiply(0.0001),
    'BLUE': s2.select('B2').multiply(0.0001)
  }).rename('EVI');

// SAVI

var savi = s2.expression(
    '1.5 * ((NIR - RED) / (NIR + RED + 0.5))', {
      'NIR': s2.select('B8').multiply(0.0001),
      'RED': s2.select('B4').multiply(0.0001),
}).rename('savi');

// NDWI (humidité)

var ndwi = s2.normalizedDifference(['B3', 'B8']).rename('NDWI');

// NDRE (qualité de la vegetation)

var ndre = image.normalizedDifference(['B8','B5']).rename('ndre');

// MSAVI (zones a vegetation clairsemee)

var msavi = image.expression(
    '(2 * NIR + 1 - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2', {
      'NIR': image.select('B8'),
      'RED': image.select('B4')
    }).rename('MSAVI');

// 5-Visualisation

var demVis = {min:0, max:3000, palette: ['black', 'white']};
var slopeVis = {min:0, max:60, palette: ['white', 'red']};
var aspectVis = {min:0, max: 360, palette: ['blue', 'green', 'yellow']};
var rgbVis = {min: 0.0, max: 3000, bands: ['B4', 'B3', 'B2']};
var ndviVis = {min:0, max:0.6, palette: [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301'
]};
var eviVis = {min :-0.1, max:0.5, palette: ['lightblue', 'darkgreen']};
var saviVis = {min:0, max:0.4, palette: ['beige', 'darkgreen']};
var ndwiVis = {min:0, max:0.4, palette: ['white', 'blue']};
var msaviVis = {min:0, max:0.6, palette: [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301'
]};
var ndreVis = {min: 0.1, max: 0.5, palette: ['440154', '3B528B', '21908C', '5DC863', 'FDE725']};

Map.addLayer(s2.clip(CSP), rgbVis, 'Image');
Map.addLayer(dem.clip(CSP), demVis, 'dem');
Map.addLayer(slope.clip(CSP), slopeVis, 'slopes');
Map.addLayer(aspect.clip(CSP), aspectVis, 'aspect');
Map.addLayer(evi.clip(CSP), eviVis, 'evi');
Map.addLayer(ndwi.clip(CSP), ndwiVis, 'ndwi');
Map.addLayer(savi.clip(CSP), saviVis, 'savi');
Map.addLayer(ndvi.clip(CSP), ndviVis, 'ndvi');
Map.addLayer(ndre.clip(CSP), ndreVis, 'ndre');
Map.addLayer(msavi.clip(CSP), msaviVis, 'msavi');

// Export vers drive

Export.image.toDrive({
  image: s2,
  description : 'origine',
  folder: 'Pastureland',
  fileNamePrefix: 'Origine',
  region:CSP,
  scale:10,
  maxPixels:1e13,
})
